<h2 align="center">Codebasics DL Course: Optuna Hyperparameter Tunning</h2>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import optuna
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

c:\Users\ahmad\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Syntetically Generated Dataset

In [2]:
# 1. Generate synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
X_train.shape

(800, 20)

In [4]:
X_train[0]

array([ 0.50363664, -1.51368248, -0.46907062,  1.90176571, -0.87064279,
        1.82004715,  1.66291365,  1.29105223, -0.16713608, -1.04718436,
        1.43003039,  0.20104766,  1.27577182, -1.13260729,  1.75008532,
       -1.4089039 ,  0.03301588, -0.80340946, -1.31410638,  1.41209637])

In [5]:
y_train[0]

np.int64(1)

In [6]:
# Convert to PyTorch tensors
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long)

### Define a Neural Network

In [7]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 2)  # Output layer for binary classification
        )
        
    def forward(self, x):
        return self.network(x)

### Perform Hyperparameter Tunning

In [8]:
# 3. Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
    hidden_dim = trial.suggest_int('hidden_dim', 16, 128)

    # Model, loss, optimizer
    model = SimpleNN(input_dim=20, hidden_dim=hidden_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    epochs = 20
    batch_size = 32
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

    # Validation accuracy
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

    accuracy = correct / total
    return accuracy

# 4. Run the Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# 5. Print the best hyperparameters
print("Best hyperparameters found:")
print(study.best_params)            

[I 2026-01-01 12:50:49,253] A new study created in memory with name: no-name-153bf485-4610-4996-b3e2-fcb1b49bbf71
C:\Users\ahmad\AppData\Local\Temp\ipykernel_3864\3446579323.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2026-01-01 12:50:51,361] Trial 0 finished with value: 0.785 and parameters: {'learning_rate': 0.036519821169105786, 'hidden_dim': 56}. Best is trial 0 with value: 0.785.
C:\Users\ahmad\AppData\Local\Temp\ipykernel_3864\3446579323.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2026-01-01 12:50:51,701]

Best hyperparameters found:
{'learning_rate': 0.0006414020257819665, 'hidden_dim': 57}
